# 3D CNN

Basic + Augmentations (invert, rotate, zoom, blur, noise)

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../..')

import datetime
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import numpy as np

from src.data import train_test_split, MRISequence
from src.model import create_model, compile_model, load_checkpoint
from src.model.training import train
from src.model.evaluation import plot_training_history, show_simple_metrics, show_metrics

In [2]:
import tensorflow as tf

# RANDOM_SEED = 4219496390001
# tf.random.set_seed(RANDOM_SEED)

print(tf.version.VERSION)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.3.1
Num GPUs Available:  1


## Setup

In [3]:
%%time

ROOT_DIR = '../../tmp'
DEFAULT_CHECKPOINT_DIRECTORY_LOCAL = os.path.join(ROOT_DIR, 'checkpoints')
DEFAULT_BCKP_CHECKPOINT_DIRECTORY_LOCAL = os.path.join(ROOT_DIR, 'bckp-checkpoints')

LOG_DIRECTORY = os.path.join(ROOT_DIR, 'logs')
CHECKPOINT_DIRECTORY = DEFAULT_CHECKPOINT_DIRECTORY_LOCAL

LOG_DIRECTORY_LOCAL = LOG_DIRECTORY
CHECKPOINT_DIRECTORY_LOCAL = CHECKPOINT_DIRECTORY

DATA_DIR_NAME = 'data-v3'
DATA_DIR = os.path.join(ROOT_DIR, DATA_DIR_NAME)

saliencies_and_segmentations_v2_path = os.path.join(ROOT_DIR, 'saliencies_and_segmentations_v2')

if not os.path.exists(CHECKPOINT_DIRECTORY):
    os.mkdir(CHECKPOINT_DIRECTORY)

if not os.path.exists(LOG_DIRECTORY):
    os.mkdir(LOG_DIRECTORY)

val = False
    
class_names = ['AD', 'CN']

# get paths to data
train_dir, test_dir, val_dir = train_test_split(
    saliencies_and_segmentations_v2_path, 
    ROOT_DIR, 
    split=(0.8, 0.15, 0.05), 
    dirname=DATA_DIR_NAME)

# set the batch size for mri seq
batch_size = 12
input_shape = (104, 128, 104, 1) # (112, 112, 105, 1)
resize_img = True
crop_img = True

# if y is one-hot encoded or just scalar number
one_hot = True

# class weightss (see analysis notebook)
class_weights = {0: 0.8072289156626505, 1: 1.3137254901960784}

# description statistics of the dataset
desc = {'mean': -3.6344006e-09, 'std': 1.0000092, 'min': -1.4982183, 'max': 10.744175}

if 'desc' not in locals():
    print('initializing desc...')
    desc = get_description(MRISequence(
        train_dir,
        64,
        class_names=class_names,
        input_shape=input_shape),
        max_samples=None)
    print(desc)


normalization={ 'type':'normalization', 'desc': desc }
# normalization={'type':'standardization', 'desc':desc }

augmentations = None
augmentations_inplace = True
# enable augmentations in mri seq (otherwise it can be enabled in dataset)
# augmentations={ 'random_swap_hemispheres': 0.5 }

# initialize sequences
print('initializing train_seq...')
train_seq = MRISequence(
    train_dir,
    batch_size,
    class_names=class_names,
    augmentations=augmentations,
    augmentations_inplace=augmentations_inplace,
    input_shape=input_shape,
    resize_img=resize_img,
    crop_img=crop_img,
    one_hot=one_hot,
    class_weights=class_weights,
    normalization=normalization)

print('initializing test_seq...')
test_seq = MRISequence(
    test_dir,
    batch_size,
    class_names=class_names,
    input_shape=input_shape,
    resize_img=resize_img,
    crop_img=crop_img,
    one_hot=one_hot,
    normalization=normalization)

if val:
    print('initializing val_seq...')
    val_seq = MRISequence(
        val_dir,
        batch_size,
        class_names=class_names,
        input_shape=input_shape,
        resize_img=resize_img,
        crop_img=crop_img,
        one_hot=one_hot,
        class_weights=class_weights,
        normalization=normalization)
else:
    print('val_seq = test_seq')
    val_seq = test_seq
    
model_key = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
log_dir = os.path.join(LOG_DIRECTORY, model_key)
print(f'log_dir: {log_dir}')

not copying files since the destination directory already exists
initializing train_seq...
initializing test_seq...
val_seq = test_seq
log_dir: ../../tmp\logs\20201218-100910
Wall time: 15.7 ms


In [4]:
# https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#class_weights
# pos / neg
initial_bias = np.log([159/243, 243/159])

model_type = '3d_cnn'
model_config = {
    'input_shape': input_shape,
    'class_names': class_names,
#     'l2_beta': 0.001,
#     'l2_beta': 0.0005,
#     'l2_beta': None,
#     'dropout': 0.10,
#     'dropout': None,
    'output_bias': initial_bias,
#     'output_bias': None,
    # https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization
    'batch_norm': False,
    'is_complex': False, # a complex layer from the paper, max batch_size is 3
}

compile_config = {
    # default is 0.001
#     'learning_rate': 0.000075,
    'learning_rate': 0.00025,
    'decay_steps': 25,
    'decay_rate': 0.96,
#     'beta_1': 0.85,
    'beta_1': 0.90,
#     'beta_2': 0.990,
    'beta_2': 0.999,
}

train_config = {
    'model_key': model_key,
    'epochs': 200,
    'patience': 25,
    'tensorboard_update_freq': 'epoch',
    'mri_tensorboard_callback': False,
    'model_checkpoint_callback': {'monitor': 'val_auc', 'mode': 'max', 'save_best_only': True},
    'early_stopping_monitor': {'monitor': 'val_auc', 'mode': 'max'},
    'augmentations': False,
    'augmentations': {
        'invert': (0.5, None),
        'rotate': (0.2, 5), # probability, degrees
        'zoom': (0.2, 0.015),
        'shear': (0, 2.5), # probability, degrees
        'blur': (0.2, 0.8),
        'noise': (0.2, 0.00025)
    },
    'batch_size': 8,
#     'model_checkpoint_callback': False,
}

## Train

In [5]:
model = create_model(model_type, model_config)
model, *_ = compile_model(model, **compile_config)
model.build(input_shape=input_shape)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 104, 128, 104, 32) 896       
_________________________________________________________________
activation (Activation)      (None, 104, 128, 104, 32) 0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 52, 64, 52, 32)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 52, 64, 52, 64)    55360     
_________________________________________________________________
activation_1 (Activation)    (None, 52, 64, 52, 64)    0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 17, 21, 17, 64)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 17, 21, 17, 128)   2

In [6]:
# load_checkpoint(model, DEFAULT_CHECKPOINT_DIRECTORY_LOCAL, '20201213-182225', 'cp-0002.ckpt')

In [7]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

print(get_available_gpus())

['/device:GPU:0']


In [8]:
%%time

model, checkpoint_dir, history = train(
    model,
    train_seq,
    test_seq,
    CHECKPOINT_DIRECTORY,
    LOG_DIRECTORY,
    DATA_DIR,
    **train_config
)

model key: 20201218-100910
checkpoint dir: ../../tmp\checkpoints\20201218-100910
log dir: ../../tmp\logs\20201218-100910
loading ../../tmp\data-v3\train_x.npy, ../../tmp\data-v3\train_y.npy...
loading ../../tmp\data-v3\val_x.npy, ../../tmp\data-v3\val_y.npy...
training...
Epoch 1/200
51/51 [==============================] - ETA: 0s - loss: 0.7333 - recall: 0.4303 - precision: 0.4303 - auc: 0.4257 - categorical_accuracy: 0.4303
Epoch 00001: val_auc improved from -inf to 0.41770, saving model to ../../tmp\checkpoints\20201218-100910\cp-0001.ckpt
51/51 [==============================] - 16s 316ms/step - loss: 0.7333 - recall: 0.4303 - precision: 0.4303 - auc: 0.4257 - categorical_accuracy: 0.4303 - val_loss: 0.7319 - val_recall: 0.3800 - val_precision: 0.3800 - val_auc: 0.4177 - val_categorical_accuracy: 0.3800
Epoch 2/200
51/51 [==============================] - ETA: 0s - loss: 0.7123 - recall: 0.4900 - precision: 0.4900 - auc: 0.4370 - categorical_accuracy: 0.4900
Epoch 00002: val_auc d

51/51 [==============================] - ETA: 0s - loss: 0.6579 - recall: 0.6045 - precision: 0.6045 - auc: 0.6338 - categorical_accuracy: 0.6045
Epoch 00016: val_auc improved from 0.49095 to 0.51405, saving model to ../../tmp\checkpoints\20201218-100910\cp-0016.ckpt
51/51 [==============================] - 15s 289ms/step - loss: 0.6579 - recall: 0.6045 - precision: 0.6045 - auc: 0.6338 - categorical_accuracy: 0.6045 - val_loss: 0.6850 - val_recall: 0.4700 - val_precision: 0.4700 - val_auc: 0.5141 - val_categorical_accuracy: 0.4700
Epoch 17/200
51/51 [==============================] - ETA: 0s - loss: 0.6464 - recall: 0.6244 - precision: 0.6244 - auc: 0.6682 - categorical_accuracy: 0.6244
Epoch 00017: val_auc did not improve from 0.51405
51/51 [==============================] - 15s 290ms/step - loss: 0.6464 - recall: 0.6244 - precision: 0.6244 - auc: 0.6682 - categorical_accuracy: 0.6244 - val_loss: 0.6841 - val_recall: 0.4300 - val_precision: 0.4300 - val_auc: 0.5064 - val_categorical_

51/51 [==============================] - ETA: 0s - loss: 0.5485 - recall: 0.7338 - precision: 0.7338 - auc: 0.8165 - categorical_accuracy: 0.7338
Epoch 00031: val_auc improved from 0.63130 to 0.64750, saving model to ../../tmp\checkpoints\20201218-100910\cp-0031.ckpt
51/51 [==============================] - 17s 332ms/step - loss: 0.5485 - recall: 0.7338 - precision: 0.7338 - auc: 0.8165 - categorical_accuracy: 0.7338 - val_loss: 0.6505 - val_recall: 0.6100 - val_precision: 0.6100 - val_auc: 0.6475 - val_categorical_accuracy: 0.6100
Epoch 32/200
51/51 [==============================] - ETA: 0s - loss: 0.5398 - recall: 0.7313 - precision: 0.7313 - auc: 0.8255 - categorical_accuracy: 0.7313
Epoch 00032: val_auc improved from 0.64750 to 0.65590, saving model to ../../tmp\checkpoints\20201218-100910\cp-0032.ckpt
51/51 [==============================] - 16s 323ms/step - loss: 0.5398 - recall: 0.7313 - precision: 0.7313 - auc: 0.8255 - categorical_accuracy: 0.7313 - val_loss: 0.6478 - val_rec

51/51 [==============================] - ETA: 0s - loss: 0.4933 - recall: 0.7637 - precision: 0.7637 - auc: 0.8642 - categorical_accuracy: 0.7637
Epoch 00046: val_auc improved from 0.70405 to 0.70430, saving model to ../../tmp\checkpoints\20201218-100910\cp-0046.ckpt
51/51 [==============================] - 16s 314ms/step - loss: 0.4933 - recall: 0.7637 - precision: 0.7637 - auc: 0.8642 - categorical_accuracy: 0.7637 - val_loss: 0.6390 - val_recall: 0.6500 - val_precision: 0.6500 - val_auc: 0.7043 - val_categorical_accuracy: 0.6500
Epoch 47/200
51/51 [==============================] - ETA: 0s - loss: 0.4866 - recall: 0.7736 - precision: 0.7736 - auc: 0.8742 - categorical_accuracy: 0.7736
Epoch 00047: val_auc improved from 0.70430 to 0.70675, saving model to ../../tmp\checkpoints\20201218-100910\cp-0047.ckpt
51/51 [==============================] - 18s 353ms/step - loss: 0.4866 - recall: 0.7736 - precision: 0.7736 - auc: 0.8742 - categorical_accuracy: 0.7736 - val_loss: 0.6390 - val_rec

Epoch 62/200
51/51 [==============================] - ETA: 0s - loss: 0.4738 - recall: 0.7910 - precision: 0.7910 - auc: 0.8811 - categorical_accuracy: 0.7910
Epoch 00062: val_auc did not improve from 0.71415
51/51 [==============================] - 17s 326ms/step - loss: 0.4738 - recall: 0.7910 - precision: 0.7910 - auc: 0.8811 - categorical_accuracy: 0.7910 - val_loss: 0.6394 - val_recall: 0.6700 - val_precision: 0.6700 - val_auc: 0.7119 - val_categorical_accuracy: 0.6700
Epoch 63/200
51/51 [==============================] - ETA: 0s - loss: 0.4679 - recall: 0.7861 - precision: 0.7861 - auc: 0.8856 - categorical_accuracy: 0.7861
Epoch 00063: val_auc did not improve from 0.71415
51/51 [==============================] - 17s 331ms/step - loss: 0.4679 - recall: 0.7861 - precision: 0.7861 - auc: 0.8856 - categorical_accuracy: 0.7861 - val_loss: 0.6395 - val_recall: 0.6800 - val_precision: 0.6800 - val_auc: 0.7139 - val_categorical_accuracy: 0.6800
Epoch 64/200
51/51 [======================

Epoch 79/200
51/51 [==============================] - ETA: 0s - loss: 0.4616 - recall: 0.7935 - precision: 0.7935 - auc: 0.8936 - categorical_accuracy: 0.7935
Epoch 00079: val_auc did not improve from 0.71475
51/51 [==============================] - 16s 319ms/step - loss: 0.4616 - recall: 0.7935 - precision: 0.7935 - auc: 0.8936 - categorical_accuracy: 0.7935 - val_loss: 0.6402 - val_recall: 0.6800 - val_precision: 0.6800 - val_auc: 0.7142 - val_categorical_accuracy: 0.6800
Epoch 80/200
51/51 [==============================] - ETA: 0s - loss: 0.4664 - recall: 0.7761 - precision: 0.7761 - auc: 0.8873 - categorical_accuracy: 0.7761
Epoch 00080: val_auc did not improve from 0.71475
51/51 [==============================] - 16s 314ms/step - loss: 0.4664 - recall: 0.7761 - precision: 0.7761 - auc: 0.8873 - categorical_accuracy: 0.7761 - val_loss: 0.6402 - val_recall: 0.6800 - val_precision: 0.6800 - val_auc: 0.7146 - val_categorical_accuracy: 0.6800
Epoch 81/200
51/51 [======================

Epoch 96/200
51/51 [==============================] - ETA: 0s - loss: 0.4732 - recall: 0.7811 - precision: 0.7811 - auc: 0.8809 - categorical_accuracy: 0.7811
Epoch 00096: val_auc did not improve from 0.71545
51/51 [==============================] - 17s 328ms/step - loss: 0.4732 - recall: 0.7811 - precision: 0.7811 - auc: 0.8809 - categorical_accuracy: 0.7811 - val_loss: 0.6403 - val_recall: 0.6800 - val_precision: 0.6800 - val_auc: 0.7145 - val_categorical_accuracy: 0.6800
Epoch 97/200
51/51 [==============================] - ETA: 0s - loss: 0.4686 - recall: 0.7935 - precision: 0.7935 - auc: 0.8853 - categorical_accuracy: 0.7935
Epoch 00097: val_auc did not improve from 0.71545
51/51 [==============================] - 16s 317ms/step - loss: 0.4686 - recall: 0.7935 - precision: 0.7935 - auc: 0.8853 - categorical_accuracy: 0.7935 - val_loss: 0.6403 - val_recall: 0.6800 - val_precision: 0.6800 - val_auc: 0.7145 - val_categorical_accuracy: 0.6800
Epoch 98/200
51/51 [======================

## Evaluation

In [9]:
%%time

plot_training_history(history)

Wall time: 152 ms


In [10]:
%%time

show_simple_metrics(model, test_seq)

loss: 0.6319835782051086
recall: 0.6800000071525574
precision: 0.6800000071525574
auc: 0.7154499888420105
categorical_accuracy: 0.6800000071525574

Wall time: 28.6 s


In [11]:
%%time

show_metrics(model, test_seq, class_names)

In binary classification, recall of the positive class is also known as “sensitivity”; recall of the negative class is “specificity”. (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)
AD
precision: 0.7884615384615384
recall: 0.6612903225806451
f1-score: 0.719298245614035
support: 62


CN
precision: 0.5625
recall: 0.7105263157894737
f1-score: 0.627906976744186
support: 38


micro avg
precision: 0.68
recall: 0.68
f1-score: 0.68
support: 100


macro avg
precision: 0.6754807692307692
recall: 0.6859083191850595
f1-score: 0.6736026111791105
support: 100


weighted avg
precision: 0.7025961538461538
recall: 0.68
f1-score: 0.6845695634434924
support: 100


samples avg
precision: 0.68
recall: 0.68
f1-score: 0.68
support: 100


F1
micro: 0.68
macro: 0.6736026111791105


accuracy_score: 0.68
sensitivity_score: 0.6612903225806451
specificity_score: 0.7105263157894737


Wall time: 26.7 s
